# Extracting Financial Statements from SEC Filings

In [65]:
api_key = "a1f46cf5ef92f8f2bc866471e25910cb8bda35a8fb2461398e8bb48af7c9485d"
xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"

In [66]:
import requests
import json
import pandas as pd
import numpy as np

# Getting the URLs and Accessing the 10-K Filings:

In [67]:
filing_10k_url = "https://www.sec.gov/ix?doc=/Archives/edgar/data/0001318605/000095017023001409/tsla-20221231.htm"
api_access_url = xbrl_converter_api_endpoint + "?htm-url=" + filing_10k_url + "&token=" + api_key

In [68]:
response = requests.get(api_access_url)
filing_json = json.loads(response.text)

## Getting Income Statement from Filing:

In [69]:
print(filing_json['StatementsOfIncome'])

{'EarningsPerShareBasic': [{'decimals': '2', 'unitRef': 'U_USDollarShare', 'period': {'startDate': '2022-01-01', 'endDate': '2022-12-31'}, 'value': '4.02'}, {'decimals': '2', 'unitRef': 'U_USDollarShare', 'period': {'startDate': '2021-01-01', 'endDate': '2021-12-31'}, 'value': '1.87'}, {'decimals': '2', 'unitRef': 'U_USDollarShare', 'period': {'startDate': '2020-01-01', 'endDate': '2020-12-31'}, 'value': '0.25'}], 'WeightedAverageNumberOfSharesOutstandingBasic': [{'decimals': '-6', 'unitRef': 'U_shares', 'period': {'startDate': '2022-01-01', 'endDate': '2022-12-31'}, 'value': '3130000000'}, {'decimals': '-6', 'unitRef': 'U_shares', 'period': {'startDate': '2021-01-01', 'endDate': '2021-12-31'}, 'value': '2959000000'}, {'decimals': '-6', 'unitRef': 'U_shares', 'period': {'startDate': '2020-01-01', 'endDate': '2020-12-31'}, 'value': '2798000000'}], 'ResearchAndDevelopmentExpense': [{'decimals': '-6', 'unitRef': 'U_USD', 'period': {'startDate': '2022-01-01', 'endDate': '2022-12-31'}, 'val

In [70]:
income_state_json = filing_json['StatementsOfIncome']

## Convert the Income Statement into a Data Frame

In [71]:
def transform_income_statement(statement_json):
    store_income_statement= {}

    # iterate over each US GAAP item in the income statement
    for item in statement_json:
        values = []
        indicies = []
        for fact in income_state_json[item]:
            
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                index = fact['period']['startDate'] + '-' + fact['period']['endDate']
                # ensure no index duplicates are created
                if index not in indicies:
                    values.append(fact['value'])
                    indicies.append(index)                    

            store_income_statement[item] = pd.Series(values, index=indicies, dtype = 'float64') 

        income_statement = pd.DataFrame(store_income_statement)
    
    
    # Transposing DF so that items are rows and individual column headers represent a date range
    return income_statement.T 

In [72]:
income_statement = transform_income_statement(income_state_json)

In [73]:
income_statement = income_statement.replace(np.nan,0)
income_statement = income_statement.style.format("{:.1f}")

# View Income Statement as a DataFrame

In [74]:
income_statement

,2020-01-01-2020-12-31,2021-01-01-2021-12-31,2022-01-01-2022-12-31
EarningsPerShareBasic,0.2,1.9,4.0
WeightedAverageNumberOfSharesOutstandingBasic,2798000000.0,2959000000.0,3130000000.0
ResearchAndDevelopmentExpense,1491000000.0,2593000000.0,3075000000.0
AutomotiveSalesRevenue,0.0,0.0,0.0
AutomotiveSales,0.0,0.0,0.0
EarningsPerShareDiluted,0.2,1.6,3.6
WeightedAverageNumberOfDilutedSharesOutstanding,3249000000.0,3386000000.0,3475000000.0
SellingGeneralAndAdministrativeExpense,3145000000.0,4517000000.0,3946000000.0
AutomotiveRegulatoryCredits,0.0,0.0,0.0
DirectCostsOfLeasedAndRentedPropertyOrEquipment,0.0,0.0,0.0


## Get Balance Sheet from URL

In [75]:
balance_sheet_json = filing_json['BalanceSheets']

## Convert the Balance Sheet into a Data Frame

In [88]:
def transform_balance_sheet(statement_json):
    store_balance_sheet = {}

    # iterate over each US GAAP item in the income statement
    for item in statement_json:
        values = []
        indicies = []
        for fact in statement_json[item]:
            
            # only consider items without segment. not required for our analysis.
            if 'segment' not in fact:
                index =  fact['period']['instant']
                # ensure no index duplicates are created
                if index in indicies:
                    continue
                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])
                    
                indicies.append(index)                    

            store_balance_sheet[item] = pd.Series(values, index=indicies, dtype = 'float64') 

        balance_sheet = pd.DataFrame(store_balance_sheet)
    
    
    # Transposing DF so that items are rows and individual column headers represent a date range
    return balance_sheet.T 

In [89]:
balance_sheet = transform_balance_sheet(balance_sheet_json)
balance_sheet = balance_sheet.replace(np.nan,0)
balance_sheet = balance_sheet.style.format("{:.1f}")

# View Balance Sheet as Data Frame

In [86]:
balance_sheet

,2020-12-31,2021-01-01,2021-12-31,2022-12-31
PreferredStockValue,0.0,0.0,0.0,0.0
CashAndCashEquivalentsAtCarryingValue,19384000000.0,0.0,17576000000.0,16253000000.0
AccountsPayableCurrent,0.0,0.0,10025000000.0,15255000000.0
MinorityInterest,0.0,0.0,826000000.0,785000000.0
LongTermDebtAndFinanceLeasesNoncurrent,9556000000.0,9775000000.0,5245000000.0,1597000000.0
CommonStockValue,0.0,0.0,3000000.0,3000000.0
DeferredCostsLeasingNetNoncurrent,0.0,0.0,0.0,0.0
ShortTermInvestments,0.0,0.0,131000000.0,5932000000.0
AccruedAndOtherCurrentLiabilities,0.0,0.0,5719000000.0,7142000000.0
CommitmentsAndContingencies,0.0,0.0,0.0,0.0


## Get Cash Flow Statement from API 

In [87]:
cash_flow_json = filing_json["StatementsOfCashFlows"]
cash_flow_json

{'ProceedsFromIssuanceOfCommonStock': [{'decimals': '-6',
   'unitRef': 'U_USD',
   'period': {'startDate': '2022-01-01', 'endDate': '2022-12-31'},
   'value': '0'},
  {'decimals': '-6',
   'unitRef': 'U_USD',
   'period': {'startDate': '2021-01-01', 'endDate': '2021-12-31'},
   'value': '0'},
  {'decimals': '-6',
   'unitRef': 'U_USD',
   'period': {'startDate': '2020-01-01', 'endDate': '2020-12-31'},
   'value': '12269000000'}],
 'InterestPaidNet': [{'decimals': '-6',
   'unitRef': 'U_USD',
   'period': {'startDate': '2022-01-01', 'endDate': '2022-12-31'},
   'value': '152000000'},
  {'decimals': '-6',
   'unitRef': 'U_USD',
   'period': {'startDate': '2021-01-01', 'endDate': '2021-12-31'},
   'value': '266000000'},
  {'decimals': '-6',
   'unitRef': 'U_USD',
   'period': {'startDate': '2020-01-01', 'endDate': '2020-12-31'},
   'value': '444000000'}],
 'ProfitLoss': [{'decimals': '-6',
   'unitRef': 'U_USD',
   'period': {'startDate': '2022-01-01', 'endDate': '2022-12-31'},
   'value

In [97]:
def transform_cash_flow_statement(statement_json):
    store_cash_flows = {}

    for item in statement_json:
        values = []
        indicies = []

        for fact in statement_json[item]:        
            # only consider items without segment.
            if 'segment' not in fact:
                # check if date instant or date range is present
                if "instant" in fact['period']:
                    index = fact['period']['instant']
                else:
                    index = fact['period']['startDate'] + '-' + fact['period']['endDate']

                # avoid duplicate indicies with same values
                if index in indicies:
                    continue
                if "value" not in fact:
                    values.append(0)
                else:
                    values.append(fact['value'])

                indicies.append(index)                    

        store_cash_flows[item] = pd.Series(values, index=indicies) 


    cash_flows = pd.DataFrame(store_cash_flows)
    return cash_flows.T

In [100]:
cash_flow_statement = transform_cash_flow_statement(cash_flow_json)
cash_flow_statement = cash_flow_statement.replace(np.nan,0)
# cash_flow_statement = cash_flow_statement.style.format("{:.1f}")

In [102]:
cash_flow_statement

,2019-12-31,2020-01-01-2020-12-31,2020-12-31,2021-01-01-2021-12-31,2021-12-31,2022-01-01-2022-12-31,2022-12-31
ProceedsFromIssuanceOfCommonStock,0,12269000000,0,0,0,0,0
InterestPaidNet,0,444000000,0,266000000,0,152000000,0
ProfitLoss,0,862000000,0,5644000000,0,12587000000,0
PaymentsToAcquirePropertyPlantAndEquipment,0,3157000000,0,6482000000,0,7158000000,0
DepreciationAmortizationAndImpairment,0,2322000000,0,2911000000,0,3747000000,0
IncreaseDecreaseInAccountsReceivable,0,652000000,0,130000000,0,1124000000,0
NoncashOrPartNoncashAcquisitionValueOfAssetsAcquired1,0,1088000000,0,2251000000,0,2148000000,0
ProceedsFromIssuanceOfDebt,0,9713000000,0,8883000000,0,0,0
IncomeTaxesPaid,0,115000000,0,561000000,0,1203000000,0
PaymentsForSolarEnergySystemsNetOfSales,0,75000000,0,32000000,0,5000000,0
